In [61]:
import pandas as pd
import numpy as np
from sklearn.feature_extraction.text import CountVectorizer
import matplotlib.pyplot as plt
import seaborn as sns
from nltk.corpus import stopwords
import re
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.linear_model import LogisticRegression
from wordcloud import WordCloud
import gensim
# from keras import layers
# from keras.models import Sequential
from sklearn.model_selection import train_test_split
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.naive_bayes import MultinomialNB
from sklearn.naive_bayes import GaussianNB


In [36]:

data = pd.read_csv('train.csv/train.csv')



data['is_toxic'] = np.where((data['toxic'] + data['severe_toxic'] + data['obscene']+ data['threat'] + data['insult'] + data['identity_hate'])!= 0, 1, 0)
# data.head()
# data[data['is_toxic'] == 1].head()

In [37]:
no_braces = data.loc[((data['comment_text'].str.contains('{', regex = False) - 1)* -1).astype(bool)]

In [39]:
# IMBALANCED LEARN STUFF
from imblearn.under_sampling import ClusterCentroids, RandomUnderSampler, NearMiss

In [40]:
X_before_vec = data.loc[:, data.columns != 'is_toxic'] 
y_sampling = data.loc[:, data.columns == 'is_toxic']

In [103]:
from sklearn.model_selection import KFold
from collections import defaultdict
from sklearn.metrics import classification_report, accuracy_score, f1_score, recall_score
def undersample(x, y, method = RandomUnderSampler(random_state = 1), return_all = True):
    if return_all:
        return method.fit_resample(x,y)
    return method.fit_resample(x,y)[0]

def transform_to_useable(x):
    tfd = TfidfVectorizer(stop_words='english',max_features=5000)
    X = tfd.fit_transform(x['comment_text'])
    y = x.loc[:, x.columns != 'comment_text']
    return X, y


def k_fold_val(X, y, models = [LogisticRegression(C = 12, max_iter = 500)], folds = 5):
    kf = KFold(n_splits = folds)
    kf.get_n_splits(X)
    results = defaultdict(list)
    for model in models:
        for cat in y:
            print(f'doing kfold for {model} on {cat}')
            for i, (train, test) in enumerate(kf.split(X)):
                y_sub = y[cat]
                model.fit(X[train], y_sub[train])
                y_pred = model.predict(X[test])

                results[(model, cat)].append({'report': classification_report(y_sub[test], y_pred),
                                             'acc':accuracy_score(y_sub[test], y_pred),
                                             'f1': f1_score(y_sub[test], y_pred),
                                             'recall': recall_score(y_sub[test], y_pred),
                                             })
    return results

def full_run(x, y, undersample_method = RandomUnderSampler(random_state = 1), combined_x_y = True, models = [LogisticRegression(C = 12, max_iter = 500)], folds = 5):
    '''
    ALL BEFORE UNDERSAMPLING
    X: the x data before vectorization
    y: the labels of data
    
    undersample_method: the way to undersample, might be able to use oversample as well? need to check
    combined_x_y: Is the data for Y inside of X, did you create a seperate classification to do undersampling
    models: the models to be tested
    folds: number of of kfolds to test
    '''
    
    if combined_x_y:
        X = undersample(x, y, undersample_method, return_all = False)
        X, y = transform_to_useable(X)
    else:
        X, y = undersample(x, y, undersample_method, return_all = True)
        X = transform_to_useable(X)
    return k_fold_val(X, y, models, folds)

In [59]:
a = undersample(X_before_vec, y_sampling, return_all = False)
X, y = transform_to_useable(a)
y = y.loc[:, y.columns != 'id']
# for col in y:
#     print(sum(y[col]))

15294
1595
8449
478
7877
1405


In [104]:
reports = full_run(X_before_vec, y_sampling)

doing kfold for LogisticRegression(C=12, max_iter=500) on id


KeyboardInterrupt: 

In [ ]:
for key, value in reports.items():
    print(key)
    for v in value:
        print(v['recall'])
    print('---------------------------------------------------------------')